In [1]:
import numpy as np
import os
from ucimlrepo import fetch_ucirepo 
import pandas as pd
import sklearn
from sklearn import linear_model
from sklearn import model_selection

import simplified_poly_eknn  
from simplified_poly_eknn import exclusive_lasso, EkNN_R, ExclusiveLassoKNNClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'ucimlrepo'

#### Test on LSVT Voice rehabitation dataset 

**Pre-processing data set**
- In this section, the data set is loaded in as a Data Frame. This Data Frame is then split into training and testing set. 

- The data is then fit into matrix to match the input of the functions. 

In [2]:
lsvt_data = pd.read_excel('lsvt/LSVT_voice_rehabilitation.xlsx')
lsvt_targets = pd.read_excel('lsvt/LSVT_voice_rehabilitation.xlsx',sheet_name=1)
lsvt_data

,Jitter->F0_abs_dif,Jitter->F0_dif_percent,Jitter->F0_PQ5_classical_Schoentgen,Jitter->F0_PQ5_classical_Baken,Jitter->F0_PQ5_generalised_Schoentgen,Jitter->F0_abs0th_perturb,Jitter->F0_CV,Jitter->F0_TKEO_mean,Jitter->F0_TKEO_std,Jitter->F0_TKEO_prc5,...,det_TKEO_std4_1_coef,det_TKEO_std4_2_coef,det_TKEO_std4_3_coef,det_TKEO_std4_4_coef,det_TKEO_std4_5_coef,det_TKEO_std4_6_coef,det_TKEO_std4_7_coef,det_TKEO_std4_8_coef,det_TKEO_std4_9_coef,det_TKEO_std4_10_coef
0,0.088112,0.041697,0.000480,-3.723304e-06,0.000422,2.458381,6.332164e-07,47.021079,1366.430390,-7.103323,...,2.527583,7.088978,19.753255,54.335046,145.528630,375.097397,921.296579,2137.079844,4697.131077,9931.208257
1,0.161798,0.057364,0.000677,5.466365e-06,0.000206,2.592066,7.228518e-07,93.557936,2582.922776,-23.284761,...,2.841881,7.977363,22.203504,60.993338,163.560972,421.010306,1036.092589,2404.072562,5284.082128,11165.095662
2,0.554508,0.642913,0.007576,-7.443871e-07,0.006488,12.691326,6.946246e-04,52.988422,466.682635,-45.308680,...,1.806103,5.078616,14.135923,38.641654,103.466808,264.654626,649.657090,1507.384591,3315.804236,6974.600636
3,0.031089,0.027108,0.000314,-2.214722e-07,0.000216,0.754288,1.868647e-07,13.982754,417.217249,-1.207741,...,1.999637,5.610448,15.626164,42.943275,115.014975,296.320795,728.284936,1689.586636,3713.818933,7851.139360
4,0.076177,0.039071,0.000302,2.732106e-05,0.001102,1.270034,4.918186e-05,56.373996,1608.317410,-3.491990,...,2.453087,6.902199,19.117609,52.715873,141.113865,363.511021,893.246151,2071.625622,4554.204815,9623.566242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,0.116214,0.070546,0.000837,2.765074e-06,0.000333,1.890006,9.844257e-07,34.083311,896.672947,-10.446397,...,2.322612,6.527285,18.210931,50.085484,134.123291,345.396264,850.942761,1973.383824,4336.099395,9158.984652
122,0.700258,0.334397,0.003959,8.297261e-06,0.001516,4.557797,1.581899e-05,104.648435,1583.166169,-97.281717,...,2.545596,7.160142,20.081389,55.178858,147.584708,379.897760,935.982559,2166.960428,4769.956102,10067.750435
123,0.072635,0.050743,0.000597,-5.277518e-06,0.000434,6.984651,4.993260e-07,21.859427,625.288493,-4.116001,...,2.189909,6.141876,17.095325,46.893015,125.687344,323.728298,795.715774,1845.609006,4056.256338,8583.121863
124,0.111362,0.054237,0.000646,-1.546671e-06,0.000277,1.935398,4.398078e-07,47.870508,1367.843467,-9.373059,...,2.539158,7.125849,19.849347,54.508453,146.094711,376.377835,926.435019,2147.499571,4717.270683,9966.759379


In [3]:
lsvt_cols = lsvt_data.columns
for i,c in enumerate(lsvt_cols):
    lsvt_data[c] = (lsvt_data[c] - lsvt_data[c].mean()) / lsvt_data[c].std()
lsvt_data

,Jitter->F0_abs_dif,Jitter->F0_dif_percent,Jitter->F0_PQ5_classical_Schoentgen,Jitter->F0_PQ5_classical_Baken,Jitter->F0_PQ5_generalised_Schoentgen,Jitter->F0_abs0th_perturb,Jitter->F0_CV,Jitter->F0_TKEO_mean,Jitter->F0_TKEO_std,Jitter->F0_TKEO_prc5,...,det_TKEO_std4_1_coef,det_TKEO_std4_2_coef,det_TKEO_std4_3_coef,det_TKEO_std4_4_coef,det_TKEO_std4_5_coef,det_TKEO_std4_6_coef,det_TKEO_std4_7_coef,det_TKEO_std4_8_coef,det_TKEO_std4_9_coef,det_TKEO_std4_10_coef
0,-0.397789,-0.429537,-0.426074,-0.192390,-0.348398,-0.184518,-0.189845,-0.329008,-0.263524,0.329039,...,0.075630,0.074287,0.069938,0.084969,0.085720,0.086476,0.068103,0.069911,0.072640,0.086308
1,-0.184727,-0.342545,-0.333477,0.391469,-0.466007,-0.179011,-0.189742,0.106345,0.482221,0.164240,...,0.945456,0.955122,0.945823,0.949890,0.966695,0.955441,0.956298,0.970696,0.970809,0.990199
2,0.950802,2.908756,2.914258,-0.003126,2.956025,0.237020,0.606656,-0.273183,-0.815095,-0.060062,...,-1.921085,-1.918990,-1.938079,-1.953623,-1.969222,-2.003805,-2.033616,-2.054567,-2.041104,-2.079571
3,-0.562675,-0.510548,-0.504203,0.030097,-0.460212,-0.254717,-0.190358,-0.638082,-0.845419,0.389083,...,-1.385474,-1.391677,-1.405366,-1.394837,-1.405033,-1.404479,-1.425260,-1.439850,-1.432051,-1.437457
4,-0.432300,-0.444122,-0.510048,1.779991,0.021915,-0.233471,-0.134125,-0.241511,-0.115240,0.365819,...,-0.130540,-0.110905,-0.157285,-0.125364,-0.129964,-0.132812,-0.148928,-0.150920,-0.146070,-0.139056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,-0.316533,-0.269353,-0.257989,0.219845,-0.396829,-0.207932,-0.189442,-0.450041,-0.551499,0.294992,...,-0.491632,-0.482633,-0.481394,-0.467055,-0.471491,-0.475659,-0.476236,-0.482370,-0.479821,-0.479388
122,1.372242,1.195700,1.211550,0.571328,0.247440,-0.098034,-0.172416,0.210097,-0.130659,-0.589380,...,0.125481,0.144846,0.187235,0.194581,0.186171,0.177329,0.181730,0.170723,0.184079,0.186333
123,-0.442544,-0.379310,-0.371019,-0.291135,-0.341825,0.001938,-0.189999,-0.564396,-0.717865,0.359464,...,-0.858892,-0.864766,-0.880187,-0.881761,-0.883631,-0.885755,-0.903536,-0.913459,-0.908044,-0.901240
124,-0.330562,-0.359912,-0.348317,-0.054099,-0.427013,-0.206062,-0.190067,-0.321062,-0.262658,0.305923,...,0.107663,0.110844,0.104287,0.107495,0.113376,0.110710,0.107859,0.105065,0.103458,0.112352


In [4]:
data_labels = []
for i in range(len(lsvt_targets['Binary class 1=acceptable, 2=unacceptable'])):
    if lsvt_targets['Binary class 1=acceptable, 2=unacceptable'][i] == 1:
        data_labels.append(0)
    else:
        data_labels.append(1)
data_labels = pd.Series(data_labels)

In [5]:
X_train, X_test, y_train ,y_test  = train_test_split(lsvt_data, data_labels, test_size=0.3)

In [6]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)


After the data is in good shape, we follows the procedures below to predict the class of the testing set.

##### Procedures for the Exclusive Lasso KNN classifier
- Extract X and Y matrix where X _(pxn)_ are compressed training predictors and label values. Y_(pxn)_ are compressed training predictors and labels values. Label values are the last row of the 2 matrices. 

- Extract `x_labels` and `y_true_labels`. `x_labels` vector is the same as `y_train` in ususal estimators. `y_true_labels` is vector is pass into the score function. Reformat the labels to `0...m` if the original ;labels are not formatted in this way. Seperate the labels from the predictors matrix for both `Y` and `X` vectors. 

- To match `sklearn` API, transpose `X` before pass into `.fit` method.

- Implement `GridSearchCV` to find best parameters. Then pass in `Y` and predict the labels for `Y`. Accuracy is provdided for the `score` function of the estimator. 

In [7]:
# generate predictor matrix X
# matrix rows = col number
# matrix column = row number 
train_cols = X_train.shape[1] 
train_rows = X_train.shape[0]
X_train_mat = np.zeros((train_cols, train_rows))
for i,col in enumerate(lsvt_cols):
    for j in range(train_rows):
        X_train_mat[i,j] = X_train[col][j]

In [8]:
# generate test sample matrix Y
# matrix rows = col number
# matrix column = row number
# y = pxn matrix where n = number of test samples 
test_cols = X_test.shape[1]  
test_rows = X_test.shape[0]
X_test_mat = np.zeros((test_cols, test_rows))
for i,col in enumerate(lsvt_cols):
    for j in range(test_rows):
        X_test_mat[i,j] = X_test[col][j]

In [9]:
# transpose X_trainimport eknn 
X_transposed = np.transpose(X_train_mat)

In [10]:
X_train_mat.shape

(310, 88)

In [11]:
train_labels = np.zeros(y_train.shape[0])
for i in range(len(y_train)):
    if y_train[i] == 1:
        train_labels[i] = 0
    else:
        train_labels[i] = 1

In [12]:
train_labels

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0.,
       1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
       0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
       1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 0.])

In [13]:
# extract labels y as (n,) matrix
test_labels = np.zeros(y_test.shape[0])
for i in range(len(y_test)):
    if y_test[i] == 1:
        test_labels[i] = 0
    else:
        test_labels[i] = 1

`GridSearchCV` of `sklearn` is utilized for finding the bets values for number fo groups, $\lambda$ and $k$.

In [ ]:
param_grid = {
    'lambda_':np.linspace(0.00000001, 1, 20) ,
    'step_size' : np.linspace(0.0001, 1, 5),
    'group_num': [i for i in range(2,11)],
    'k': [3, 5, 7,10]
}

# Create the custom classifier instance
clf = simplified_poly_eknn.ExclusiveLassoKNNClassifier()

# Use GridSearchCV to find the best hyperparameters
grid_search = model_selection.GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')  # the scoring parameter can be changed
grid_search.fit(X_transposed, train_labels)# fit training data

# Best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_


In [ ]:
# X_test_mat = X_test_mat [:, :1]
# test_labels = test_labels[:1]
# X_test_mat, test_labels

In [14]:
clf = simplified_poly_eknn.ExclusiveLassoKNNClassifier(group_num=2, k=3, lambda_=1e-08, step_size=0.1)
clf.fit(X_transposed, train_labels)
score_lsvt = clf.score(X_test_mat,test_labels)
score_lsvt

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
-0.0026462453457351464 0.0
0.06823058921415469 0.06867147055403945
-0.2965286570132857 0.0
0.3332624382238385 0.32935160884688874
0.055893706838810425 0.0663513478021497
-0.05204995656474001 0.0
-0.036632180156005556 0.0
0.1669665316303342 0.05068853485396451
0.028097038838662523 0.0
0.08617381617852848 0.11837012297045503
0.15731675145450802 0.14872043212003955
-0.014067008475347004 0.0
0.22445956725486935 0.17650520961760824
0.25896766637896806 0.

0.6842105263157895

In [ ]:
lambda_ = best_params['lambda_']
group_num = best_params['group_num']
k = best_params['k']


In [ ]:
print("Accuracy score of classifying LSVT data set:", round(score_lsvt*100,2))

The accuracy of this model is 47.36, which is low compared to the result in the paper which is 83.25. However, this might be due to the way we group the data points. In this experiment, data are evenly group corresponding to the number of group passed in (_refer to `eknn.py` file to understand how to data points are grouped_). Hence, this can significantly affect the result of the classification. In order to see if this is the case, I run the classifier on one more data set below. 

In [ ]:
preds= clf.predict(X_test_mat)

In [ ]:
preds, test_labels

Result of the classification is printed out along with the true labels of the testing set to clearly see the performace of the proposed algorithm.

#### Test on Sonars dataset

- Arcene data set are added in order to see the performance of the algorithm on new data set.

- Similarly, data are shpaed into matrices format before being passed into the functions

In [16]:
# fetch dataset 
connectionist_bench_sonar_mines_vs_rocks = fetch_ucirepo(id=151) 
  
# data (as pandas dataframes) 
sonars_data = connectionist_bench_sonar_mines_vs_rocks.data.features 
sonars_targets = connectionist_bench_sonar_mines_vs_rocks.data.targets 

ModuleNotFoundError: No module named 'ucimlrepo'

In [16]:
sonars_cols = sonars_data.columns
for i, c in enumerate(sonars_cols):
    sonars_data[c] = (sonars_data[c] - sonars_data[c].mean()) / sonars_data[c].std()
sonars_data

NameError: name 'sonars_data' is not defined

In [17]:
X_train, X_test, y_train, y_test = train_test_split(sonars_data, sonars_targets, test_size=0.33, random_state=42)

NameError: name 'sonars_data' is not defined

In [18]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [19]:
# generate predictor matrix X
# matrix rows = col number
# matrix column = row number 
train_cols = X_train.shape[1] 
train_rows = X_train.shape[0]
X = np.zeros((train_cols, train_rows))
for i,col in enumerate(sonars_cols):
    for j in range(train_rows):
        X[i,j] = X_train[col][j]

NameError: name 'sonars_cols' is not defined

In [ ]:
# generate predictor matrix X
# matrix rows = col number
# matrix column = row number 
test_cols = X_test.shape[1] 
test_rows = X_test.shape[0]
X_test_mat = np.zeros((test_cols, test_rows))
for i,col in enumerate(sonars_cols):
    for j in range(test_rows):
        X_test_mat[i,j] = X_test[col][j]

In [ ]:
x_labels = np.zeros(y_train.shape[0])
for i in range(len(y_train["class"])):
    if y_train["class"][i] == "R":
        x_labels[i] = 0
    else:
        x_labels[i] = 1

In [ ]:
test_labels = np.zeros(y_test.shape[0])
for i in range(len(y_test["class"])):
    if y_test["class"][i] == "R":
        test_labels[i] = 0
    else:
        test_labels[i] = 1

In [ ]:
y_test = np.reshape(y_test, y_test.shape[0])
y_train = np.reshape(y_train, y_train.shape[0])

In [ ]:
X_train_transposed = np.transpose(X)

`GridSearchCV` is implemented to find best tuning parameters.

In [ ]:
param_grid = {
    'lambda_': np.logspace(-4, -0.05, 20),
    'group_num': [i for i in range(2, 11,1)],
    'step_size' : np.linspace(0.0001, 1, 5),
    'k': [3, 5, 7, 10]
}

# Create the custom classifier instance
clf = simplified_poly_eknn.ExclusiveLassoKNNClassifier()

# Use GridSearchCV to find the best hyperparameters
grid_search = model_selection.GridSearchCV(clf, param_grid, cv=10)  # the scoring parameter can be changed
grid_search.fit(X_train_transposed, y_train)# fit training data

# Best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

In [ ]:
best_params

In [ ]:
y_test

In [ ]:
X_train_transposed.shape # groups_vect has correct dimension

In [ ]:
clf = simplified_poly_eknn.ExclusiveLassoKNNClassifier(group_num=2, k=3, lambda_=0.0001, step_size=0.001)
clf.fit(X_train_transposed, x_labels)
preds = clf.predict(X_test_mat)
score_sonars = clf.score(X_test_mat, test_labels)
score_sonars

In [ ]:
print("Accuracy score of classifying Sonar data set:", round(score_sonars*100,2))

For this dataset, the accuracy score of hte classifier is 71.01 which is pretty close to the score of 74.63 in the paper. The small difference might be due to the way we group the daat points. More research on how we group data points might help obtain similar classification result. 

#### Test on vehicle data - data with multiple (more than) class labels

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
statlog_vehicle_silhouettes = fetch_ucirepo(id=149) 
  
# data (as pandas dataframes) 
vehicle_data = statlog_vehicle_silhouettes.data.features 
targets = statlog_vehicle_silhouettes.data.targets  


In [ ]:
# vehicle
null_data = vehicle_data[vehicle_data.isnull().any(axis=1)]
vehicle_data.drop(index=752,inplace=True)

In [ ]:
targets.drop(index=752, inplace=True)

In [ ]:
vehicle_data.reset_index(drop=True)
targets.reset_index(drop=True)

In [ ]:
vehicle_cols =vehicle_data.columns
for i,c in enumerate(vehicle_cols):
    vehicle_data[c] = (vehicle_data[c] - vehicle_data[c].mean()) / vehicle_data[c].std()
vehicle_data

In [ ]:
targets['class'].unique()

In [ ]:
labels=pd.Series(len(targets['class']))
for i,v in enumerate(targets['class']):
    if v == 'van':
        labels[i] = 0
    elif v == 'saab':
        labels[i] = 1
    elif v == 'bus':
        labels[i] = 2
    elif v == 'opel':
        labels[i] = 3
    else:
        labels[i] = 4
labels

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(vehicle_data, labels, test_size=0.33, random_state=42)

In [ ]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
predictors_vehicles = X_train.columns
predictors_vehicles

In [ ]:
# generate predictor matrix X
# matrix rows = col number
# matrix column = row number 
train_cols = X_train.shape[1] 
train_rows = X_train.shape[0]
X_train_mat = np.zeros((train_cols, train_rows))
for i,col in enumerate(predictors_vehicles):
    for j in range(train_rows):
        X_train_mat[i,j] = X_train[col][j]

In [ ]:
# generate predictor matrix X
# matrix rows = col number
# matrix column = row number 
test_cols = X_test.shape[1] 
test_rows = X_test.shape[0]
X_test_mat = np.zeros((test_cols, test_rows))
for i,col in enumerate(predictors_vehicles):
    for j in range(test_rows):
        X_test_mat[i,j] = X_test[col][j]

In [ ]:
X_train_transposed = np.transpose(X_train_mat)

In [ ]:
X_train_transposed.shape

In [ ]:
param_grid = {
    'lambda_': np.logspace(-4, -0.5, 30),
    'group_num': [i for i in range(2, 11,1)],
    'k': [3, 5, 7, 10,11]
}

# Create the custom classifier instance
clf = simplified_poly_eknn.ExclusiveLassoKNNClassifier()

# Use GridSearchCV to find the best hyperparameters
grid_search_3 = model_selection.GridSearchCV(clf, param_grid, cv=10)  # the scoring parameter can be changed
grid_search_3.fit(X_train_transposed, y_train)# fit training data

# Best parameters and best score
best_params = grid_search_3.best_params_
best_score = grid_search_3.best_score_

In [ ]:
lambda_ = best_params['lambda_']
group_num = best_params['group_num']
k = best_params['k']
best_params

In [ ]:
clf = simplified_poly_eknn.ExclusiveLassoKNNClassifier(group_num=2, k=3, lambda_=1e-07, step_size=0.001)
clf.fit(X_train_transposed, y_train)
score_lsvt = clf.score(X_test_mat, y_test)
score_lsvt

## Test Iosnoshere

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
ionosphere = fetch_ucirepo(id=52) 
  
# data (as pandas dataframes) 
ionosphere_data = ionosphere.data.features 
target = ionosphere.data.targets 

In [ ]:
target["Class"].unique()

In [ ]:
ionosphere_data.drop('Attribute2', axis=1, inplace=True)

In [ ]:
ionosphere_cols = ionosphere_data.columns
for i, c in enumerate(ionosphere_cols):
    ionosphere_data[c] = (ionosphere_data[c] - ionosphere_data[c].mean()) / ionosphere_data[c].std()
ionosphere_data

In [ ]:
# check for missing values
null_data = ionosphere_data[ionosphere_data.isnull().any(axis=1)]
null_data 

In [ ]:
# change values for class labels [g:0, b:1]
labels=pd.Series(len(target['Class']))
for i,v in enumerate(target['Class']):
    if v == 'g':
        labels[i] = 0
    elif v == 'b':
        labels[i] = 1
labels

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ionosphere_data, labels, test_size=0.33, random_state=42)

In [ ]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
predictors_ionosphere = X_train.columns
# generate predictor matrix X
# matrix rows = col number
# matrix column = row number 
train_cols = X_train.shape[1] 
train_rows = X_train.shape[0]
X_train_mat = np.zeros((train_cols, train_rows))
for i,col in enumerate(predictors_ionosphere):
    for j in range(train_rows):
        X_train_mat[i,j] = X_train[col][j]

In [ ]:
# generate predictor matrix X
# matrix rows = col number
# matrix column = row number 
test_cols = X_test.shape[1] 
test_rows = X_test.shape[0]
X_test_mat = np.zeros((test_cols, test_rows))
for i,col in enumerate(predictors_ionosphere):
    for j in range(test_rows):
        X_test_mat[i,j] = X_test[col][j]

In [ ]:
X_train_transposed = np.transpose(X_train_mat)

In [ ]:
param_grid = {
    'lambda_': np.logspace(-4, -0.5, 30),
    'group_num': [i for i in range(2, 11,1)],
    'k': [3, 5, 7, 10,11]
}

# Create the custom classifier instance
clf = simplified_poly_eknn.ExclusiveLassoKNNClassifier()

# Use GridSearchCV to find the best hyperparameters
grid_search_3 = model_selection.GridSearchCV(clf, param_grid, cv=10)  # the scoring parameter can be changed
grid_search_3.fit(X_train_transposed, y_train)# fit training data

# Best parameters and best score
best_params = grid_search_3.best_params_
best_score = grid_search_3.best_score_

In [ ]:
lambda_ = best_params['lambda_']
group_num = best_params['group_num']
k = best_params['k']
best_params

In [ ]:
clf = simplified_poly_eknn.ExclusiveLassoKNNClassifier(group_num=2, k=3, lambda_=0.00001, step_size=0.001)
clf.fit(X_train_transposed, y_train)
score_ionoshere = clf.score(X_test_mat, y_test)
score_ionoshere

## Test Wine data

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
wine = fetch_ucirepo(id=109) 
  
# data (as pandas dataframes) 
wine_data = wine.data.features 
targets = wine.data.targets 

In [ ]:
targets["class"].unique()

In [ ]:
# check for missing values
null_data = wine_data[wine_data.isnull().any(axis=1)]
null_data 

In [ ]:
wine_cols = wine_data.columns
for i,c in enumerate(wine_cols):
    wine_data[c] = (wine_data[c] - wine_data[c].mean()) / wine_data[c].std()
wine_data

In [ ]:
# change values for class labels [g:0, b:1]
labels=pd.Series(len(targets['class']))
for i,v in enumerate(targets['class']):
    if v == 1:
        labels[i] = 0
    elif v == 2:
        labels[i] = 1
    else: labels[i] = 2
labels

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(wine_data, labels, test_size=0.33, random_state=42)

In [ ]:
X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [ ]:
predictors_wine = X_train.columns
# generate predictor matrix X
# matrix rows = col number
# matrix column = row number 
train_cols = X_train.shape[1] 
train_rows = X_train.shape[0]
X_train_mat = np.zeros((train_cols, train_rows))
for i,col in enumerate(predictors_wine):
    for j in range(train_rows):
        X_train_mat[i,j] = X_train[col][j]

In [ ]:
# generate predictor matrix X
# matrix rows = col number
# matrix column = row number 
test_cols = X_test.shape[1] 
test_rows = X_test.shape[0]
X_test_mat = np.zeros((test_cols, test_rows))
for i,col in enumerate(predictors_wine):
    for j in range(test_rows):
        X_test_mat[i,j] = X_test[col][j]

In [ ]:
X_train_transposed = np.transpose(X_train_mat)

In [ ]:
param_grid = {
    'lambda_': np.logspace(-4, -0.5, 30),
    'group_num': [i for i in range(2, 11,1)],
    'k': [3, 5, 7, 10,11]
}

# Create the custom classifier instance
clf = simplified_poly_eknn.ExclusiveLassoKNNClassifier()

# Use GridSearchCV to find the best hyperparameters
grid_search_3 = model_selection.GridSearchCV(clf, param_grid, cv=10)  # the scoring parameter can be changed
grid_search_3.fit(X_train_transposed, y_train)# fit training data

# Best parameters and best score
best_params = grid_search_3.best_params_
best_score = grid_search_3.best_score_

In [ ]:
lambda_ = best_params['lambda_']
group_num = best_params['group_num']
k = best_params['k']
best_params

In [ ]:
clf = simplified_poly_eknn.ExclusiveLassoKNNClassifier(group_num=2, k=3, lambda_=0.0001, step_size=0.001)
clf.fit(X_train_transposed, y_train)
score_wine = clf.score(X_test_mat, y_test)
score_wine

In [ ]:
# plot correlation matrix graph


#### Uitilize unsupervised learning to group data points